## Harnessing the Power of ChatGPT and SAP HANA Cloud for Next-level Data Analytics!

In [8]:
import time
from json import load
from hana_ml import dataframe
from pathlib import Path
from pyChatGPT import ChatGPT as cGPT

In [5]:
class Chatbot:
    """
    Chatbot class contains the get_session_token and chatbot_interface functions as methods.
    The session_token variable is now an instance variable, and the api variable is created
    in the constructor. The chatbot_interface method uses self.api instead of the global
    variable api, and the get_session_token method now uses self instead of the global variable
    session_token.

    This way, you can create multiple instances of the Chatbot class, each with its own session
    token and API object.
    """

    def __init__(self):
        """
        Define chat GPT behavior as SQL Console for returning
        SQL statements only. Otherwise, some further work must
        must be done before send to SAP HANA.
        This text was based on some very good community contributions 
        here: https://github.com/f/awesome-chatgpt-prompts.
        """
        self.bot_behavior = self.get_session_token("behavior.txt")

        """
        Get session token obtained from cookies, check file.
        Set api from pyChatGPT package based on that token.
        More information on how to get it: 
        https://www.npmjs.com/package/chatgpt?activeTab=readme
        """
        self.session_token = self.get_session_token("session_token.txt")
        self.api = cGPT(self.session_token)

    def get_session_token(self, file_name):

        file_path = Path(file_name)

        if not file_path.is_file():
            print("Error: {} not found".format(file_path))
            return None
        try:
            with open(file_path, "r") as file:
                session_token = file.read().strip()
        except:
            print("Error: Unable to read {}".format(file_path))
            return None

        return session_token

    def create_interaction(self, prompt, script=False):
        def set_query(string):
            string = string.replace("`", "")
            string = string.replace("\\", "")
            string = string.replace("\n", " ")
            return string

        if not script:
            response = self.api.send_message(prompt)
            message = response["message"]

        else:
            response = self.api.send_message(self.bot_behavior + prompt)
            message = set_query(response["message"])
        
        self.response = response
        return message

In [ ]:
class SAPHANA:
    def __init__(self, connection_file):
        self.connection_file = connection_file
        self.conn = None

    def get_connection(self):
        """
        This function uses the settings provided in the access
        file for connecting to SAP HANA database.
        """
        start = time.time()
        with open(self.connection_file) as file:
            conn_properties = load(file)
            self.conn = dataframe.ConnectionContext(
                conn_properties["address"],
                conn_properties["port"],
                conn_properties["user"],
                conn_properties["password"],
            )

            msg = (
                "INFO: Connection successfully established to SAP HANA."
                if self.conn.connection.isconnected() == True
                else "ERROR: Not possible to connect. Check access file information."
            )
            print("{}".format(msg))
            end = time.time()
            print("      Elapsed time: {:.2f} seconds".format(end - start))
            print("\033[92m      MESSAGE: Logged with connection ID {} in schema {}.\033[00m".format(
                self.conn.get_connection_id(),
                self.conn.get_current_schema())
            )
    
    def connect(self):
        self.get_connection()
        return self.conn

    def get_connection_id(self):
        return self.conn.get_connection_id()

    def get_current_schema(self):
        return self.conn.get_current_schema()
    
    def close_connection(self):
        self.conn.close()


## Instantiate the class

In [6]:
# Instanciate Chatbot Class
bot = Chatbot()

## Connect to SAP HANA

In [ ]:
conn = SAPHANA('access_file.json')

# TRY IT OUT!

In [ ]:
response = bot.create_interaction(input(), script=False)

In [ ]:
print(response)